# LA County VSAP technology issues

### Load python tools

In [142]:
import pandas as pd
import geopandas as gpd
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
# alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
plt.rcParams['figure.figsize'] = (16,8)
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.float_format = '{:.0f}'.format

### Read data downloaded from RRCC office

In [143]:
src = pd.read_csv('input/cpra/tickets.csv')

In [144]:
src.columns = src.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [145]:
src.head()

,incident_id,priority,status,vcid,site_name,category,bmd_related_y/n,total_bmds,bad_bmds,gear_problem_y/n,subcategory,description,created_date_time,created_by,assigned_to,assigned_team,primary_config_asset_tag,primary_config_lac_asset_type,call_source,comments,incident_childid,incident_type,last_modified_date_time,primary_config_serial_number,primary_config_type,last_modified_by,location,bam_threshold,bmd_threshold,epollbook_threshold,status2,category_description,category_description_de_de,category_description_en_us,category_description_es_es,category_description_fr_fr,category_description_pt_br,incident_category,incident_category_de_de,incident_category_en_us,incident_category_es_es,incident_category_fr_fr,incident_category_pt_br,service,service_status,specifics_name,visible_in_portal,config_item_type,default_team,closed_by
0,105423,2,Resolved,414,Rivera Park Comm Recreation Cntr,Miscellaneous,NaN,nan,nan,NaN,No Site Access,CB NUMBER: personal\nSITE ID#: \nASSET TAG:\nS...,2/22/2020 7:06 AM,Raven Mcbride (c198168),NaN,Tech Support Dispatch (0),nan,NaN,Phone,nan,nan,Incident,3/3/2020 8:31 AM,NaN,NaN,Cherwell Services,nan,25,25,25,Active,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN
1,105424,1,Resolved,13444,LA County Fire Station 129,Hardware,NaN,nan,nan,NaN,Missing ePollbook,Janet Exie\n562-410-5023\nVCID 13444\nCustomer...,2/22/2020 7:15 AM,Miguel Penaloza (c197949),NaN,Tech Support Dispatch (0),nan,NaN,Phone,nan,nan,Incident,2/29/2020 6:16 PM,NaN,NaN,Thomas Duffey (c198556),nan,25,25,25,Active,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Tech Support Dispatch (0),NaN
2,105425,1,Resolved,13665,The Light of the World Church,Missing Equipment,NaN,nan,nan,NaN,All Equipment has not been delievered,No BMD at location they only have 2 epoll book...,2/22/2020 7:18 AM,Julia Navarro,NaN,Tech Support Dispatch (0),nan,NaN,Phone,nan,nan,Incident,3/3/2020 8:31 AM,NaN,NaN,Cherwell Services,nan,25,25,25,Active,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN
3,105426,1,Resolved,11809,Smith Park,Missing Equipment,NaN,nan,nan,NaN,Surge Protector/Power Strip,CB NUMBER: 562.519.1638\nSITE ID#: 11809\nASSE...,2/22/2020 7:19 AM,Raven Mcbride (c198168),NaN,Tech Support Dispatch (0),nan,NaN,Phone,nan,nan,Incident,3/3/2020 8:31 AM,NaN,NaN,Cherwell Services,nan,25,25,25,Active,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN
4,105435,1,Resolved,11854,Rolling Hills City Hall,Service Request,NaN,nan,nan,NaN,Account Creation,Donna Jones \n310 714 0282\nVCID 11854\n \ncan...,2/22/2020 7:28 AM,Miguel Penaloza (c197949),NaN,Tech Support Dispatch (0),nan,NaN,Phone,nan,nan,Service Request,3/3/2020 8:31 AM,NaN,NaN,Cherwell Services,nan,25,25,25,Active,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Tech Support Dispatch (0),NaN


In [146]:
src[['incident_id','priority','vcid','primary_config_asset_tag', 'incident_childid','comments']] \
= src[['incident_id','priority','vcid','primary_config_asset_tag', 'incident_childid', 'comments']].astype(str)

In [147]:
src = src.drop(columns=['category_description', 'category_description_de_de', 'category_description_en_us', \
         'category_description_es_es', 'category_description_fr_fr', 'category_description_pt_br', \
         'incident_category', 'incident_category_de_de', 'incident_category_en_us', \
         'incident_category_es_es', 'incident_category_fr_fr', 'incident_category_pt_br',\
        'location', 'bam_threshold', 'bmd_threshold', 'epollbook_threshold', 'service',\
        'service_status', 'specifics_name', 'visible_in_portal', 'config_item_type', 'config_item_type'])

In [148]:
src.vcid = src.vcid.astype(str).str.replace('.0', '', regex=False)

In [149]:
src = src.dropna(how='all')
src = src.fillna('')

In [153]:
bmd = src[src['bmd_related_y/n'] == 'Y']

In [154]:
len(bmd)

1542

In [160]:
bmd.head()

,incident_id,priority,status,vcid,site_name,category,bmd_related_y/n,total_bmds,bad_bmds,gear_problem_y/n,subcategory,description,created_date_time,created_by,assigned_to,assigned_team,primary_config_asset_tag,primary_config_lac_asset_type,call_source,comments,incident_childid,incident_type,last_modified_date_time,primary_config_serial_number,primary_config_type,last_modified_by,status2,default_team,closed_by
32,105529,4,Resolved,4743,Pico Park Community Center,Error Code,Y,,1,,715,Stacey Tinajero (e655832)\n \nPhone 5624105247...,2/22/2020 8:32 AM,Diego Corral (c197909),,Smartmatic Support (Level 2),124808.0,BMD,Phone,nan,nan,Incident,3/3/2020 8:31 AM,CNGBMDAA01008917,Config - System,Cherwell Services,Active,Tech Support Call Center (0),
56,105593,4,Resolved,11801,H&H Jivalagian Youth Center,Hardware,Y,,1,,Printer Jams Repeatedly,Asset Tag 121949\nPrinter is having Paper JAM....,2/22/2020 9:09 AM,Karanpreet Singh (e644483),,Field Support Technician (1),121949.0,BMD,Phone,nan,nan,Incident,3/3/2020 8:31 AM,CNGBMDAA03023177,Config - System,Cherwell Services,Active,,
67,105632,3,Resolved,4743,Pico Park Community Center,Hardware,Y,,2,,Poor Print quality,Stacey Tinajero (e655832)\n562-410-5247\n4743\...,2/22/2020 9:24 AM,Thomas Duffey (c198556),,Tech Support Dispatch (0),nan,,Phone,nan,nan,Incident,3/3/2020 8:31 AM,,,Cherwell Services,Active,Tech Support Call Center (0),
68,105633,3,Resolved,11801,H&H Jivalagian Youth Center,Error Code,Y,,1,,202,Liisa Lopez (id)632228\n \nBMD jamming needs t...,2/22/2020 9:25 AM,Christopher Liddell (c197934),,Tech Support Dispatch (0),nan,,Phone,nan,nan,Incident,3/3/2020 8:31 AM,,,Cherwell Services,Active,Tech Support Call Center (0),
71,105646,4,Resolved,13667,Fine Gold Manor,Error Code,Y,10,1,,501,(562) 650-1841\nVote ctr size: small 10 bmd\n ...,2/22/2020 9:32 AM,Thomas Price (c198083),,Field Support Technician (1),nan,,Phone,nan,nan,Incident,3/3/2020 8:31 AM,,,Cherwell Services,Active,Tech Support Call Center (0),
